# Using Linear Regression to Predict Canine Intelligence
This notebook utilizes the dog intelligence dataset hosted on Kaggle and provided by the American Kennel Club in order to estimate dog intelligence using a linear regression model.

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/canine-intelligence-and-size/AKC Breed Info.csv
/kaggle/input/canine-intelligence-and-size/dog_intelligence.csv


In [5]:
# load data
breed_info = pd.read_csv('/kaggle/input/canine-intelligence-and-size/AKC Breed Info.csv')
breed_info.head()

,index,Breed,height_low_inches,height_high_inches,weight_low_lbs,weight_high_lbs
0,0,Akita,26,28,80,120
1,1,Anatolian Sheepdog,27,29,100,150
2,2,Bernese Mountain Dog,23,27,85,110
3,3,Bloodhound,24,26,80,120
4,4,Borzoi,26,28,70,100


In [3]:
dog_intel = pd.read_csv('/kaggle/input/canine-intelligence-and-size/dog_intelligence.csv')
dog_intel.head()

,index,Breed,Classification,obey,reps_lower,reps_upper
0,0,Border Collie,Brightest Dogs,95%,1,4
1,1,Poodle,Brightest Dogs,95%,1,4
2,2,German Shepherd,Brightest Dogs,95%,1,4
3,3,Golden Retriever,Brightest Dogs,95%,1,4
4,4,Doberman Pinscher,Brightest Dogs,95%,1,4


In [4]:
dog_intel.dtypes

index              int64
Breed             object
Classification    object
obey              object
reps_lower         int64
reps_upper         int64
dtype: object